Import libraries

In [4]:
import gspread as gs
import pandas as pd

Authorization (maybe need to do this only once?)

In [5]:
gc = gs.oauth()

Combine CDFA plating data sheets

In [60]:
cdfa_plating = gc.open("CDFA - Plating")
plating = []
plating_columns = ['catalog', 'well', 'plate', 'preservation', 
                   'body_part', 'flagged', 'notes']

for worksheet in cdfa_plating.worksheets():
    ws = worksheet.title
    if "CDFA" in ws and ws != "CDFA_P2_transfer":
        records = worksheet.get_all_records()
        df = pd.DataFrame(records)
        df = df[plating_columns]
        plating.append(df)

Combine CDFA metadata

In [57]:
cdfa_meta = gc.open("CDFA - Metadata")
meta = []
meta_columns = ['Catalog','Order','Family','Genus', 'Species','Event Date']
for worksheet in cdfa_meta.worksheets():
    ws = worksheet.title
    if "CDFA" in ws and ws != "CDFA":
        records = worksheet.get_all_records()
        df = pd.DataFrame(records)
        df = df[meta_columns]
        meta.append(df)

Merge all dataframes in each list of dataframes

In [61]:
plating_merged = pd.concat(plating, ignore_index = True)
meta_merged = pd.concat(meta, ignore_index = True)

Needed to change the capitals in the metadata

In [74]:
meta_merged.columns = [x.lower() for x in list(meta_merged.columns)]

Index(['catalog', 'order', 'family', 'genus', 'species', 'event date'], dtype='object')


Merging the CDFA data

In [126]:
cdfa_merged = plating_merged.join(meta_merged.set_index('catalog'), 
                                  on = 'catalog', how = 'inner')
cdfa_merged['plate'] = 'CDFA_' + cdfa_merged['plate']

In [127]:
cdfa_merged['plate']

0        CDFA_P1
1        CDFA_P1
2        CDFA_P1
3        CDFA_P1
4        CDFA_P1
          ...   
2009    CDFA_P21
2010    CDFA_P21
2011    CDFA_P21
2012    CDFA_P21
2013    CDFA_P21
Name: plate, Length: 1984, dtype: object

Now to get CAS data

In [105]:
cas_plating = gc.open("Plating")

In [106]:
cas_plating = pd.DataFrame(cas_plating.get_worksheet(0).get_all_records())

In [112]:
serpa = gc.open("serpa_metadata")
cas_meta = pd.DataFrame(serpa.get_worksheet(0).get_all_records())

In [136]:
cas_merged = cas_plating.join(cas_meta.set_index('catalog'), 
                              on = 'catalog', how = 'inner')

In [138]:
plating_columns = ['catalog', 'well', 'plate', 'preservation', 
                   'body_part', 'flagged', 'notes']
meta_columns = ['order', 'family', 'genus', 'species', 'event date']
all_cols = plating_columns + meta_columns
print(all_cols)

['catalog', 'well', 'plate', 'preservation', 'body_part', 'flagged', 'notes', 'order', 'family', 'genus', 'species', 'event date']


In [139]:
cas_merged = cas_merged[all_cols]

In [142]:
df_concat = pd.concat([cas_merged, cdfa_merged], ignore_index = True)

Download the extraction and PCR data

In [143]:
extractions = gc.open("Extractions")
extract_data = pd.DataFrame(extractions.get_worksheet(0).get_all_records())
extract_quant = pd.DataFrame(extractions.get_worksheet(1).get_all_records())

pcr = gc.open("PCR")
pcr_success = pd.DataFrame(pcr.get_worksheet(3).get_all_records())


In [153]:
lab_df = extract_data.merge(extract_quant.set_index(['catalog', 'plate', 'well']), 
                            on = ['catalog', 'plate', 'well'], how = 'left')

pcr_success = pcr_success[pcr_success['catalog'] != ""]
lab_df2 = lab_df.merge(pcr_success.set_index(['catalog', 'plate', 'well']), 
                            on = ['catalog', 'plate', 'well'], how = 'left')